# GOURMET GUIDE

### Foundation Project 1: Food Recommendation Model

#### Group 6

* Akshay Ramdev - 12220032
* Charanjeet Singh - 12220064
* Pooja Nilesh Doshi - 12220028
* Snigdha Debashis Bhattacharjee - 12220067
* Vinayak Dave - 12220047

In [1]:
#!pip install threadpoolctl==3.1.0
#!pip install yellowbrick
#!pip install pca
#!pip install spacy==3.5.2
#!python -m spacy download en

In [170]:
import json
import string
import spacy
import matplotlib as m
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import os
import re
import pandas as pd
from pandas import DataFrame
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import threadpoolctl
import warnings
warnings.filterwarnings("ignore")
#from pca import pca
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [171]:
train_df = pd.read_excel(r"C:\Users\SNIGGDHA\FP\zomato_train_df.xlsx")

In [172]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6305 entries, 0 to 6304
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    6305 non-null   int64  
 1   rest_id       6305 non-null   int64  
 2   url           6305 non-null   object 
 3   name          6305 non-null   object 
 4   online_order  6305 non-null   bool   
 5   book_table    6305 non-null   bool   
 6   rate          6305 non-null   float64
 7   votes         6305 non-null   int64  
 8   location      6305 non-null   object 
 9   rest_type     6305 non-null   object 
 10  dish_liked    6305 non-null   object 
 11  cuisines      6305 non-null   object 
 12  cost          6305 non-null   float64
 13  reviews_list  6263 non-null   object 
dtypes: bool(2), float64(2), int64(3), object(7)
memory usage: 603.5+ KB


In [173]:
#Deleting Unnnecessary Columns
train_df = train_df.drop(['Unnamed: 0', 'url'],axis=1)

In [174]:
train_df.head(10)

,rest_id,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list
0,22393,Brews N Bites,True,False,2.7,50,Bannerghatta Road,Cafe,Thin Crust Pizza,Finger Food,650.0,worst food ever ordered baked veg pasta 13 inc...
1,21651,Panchami Garden,False,False,3.6,56,Bannerghatta Road,Quick Bites,"Masala Dosa, Idli",North Indian,400.0,food okayish nothing special service quick hyg...
2,25447,Sherlocks Pub,True,True,3.8,360,Kalyan Nagar,Casual Dining,"Chips, Cocktails, Veg Platter, Murgh Platter, ...",Chinese,1.3,todaynnwe actually wanted rooftop unfortunatel...
3,15369,Marzipan Cafe Bakery,True,False,4.2,432,Ulsoor,Bakery,"Coffee, Cheesecake, Chicken Baguette, Pasta, M...",Bakery,700.0,absolutely love ambience cute cozy vibe place ...
4,48954,Savoury Restaurant,True,False,2.9,833,Sarjapur Road,Casual Dining,"Kerala Parotta, Jumbo Shawarma, Dragon Chicken...",Arabian,800.0,im great fan savoury restaurant dish prefer ta...
5,23594,Once Upon A Rooftop,False,True,4.3,1246,Jayanagar,Bar,"Pizza, Cocktails, Pasta, Churros, Mocktails, M...",Thai,1.0,perfect place hang friendsnas name says roofto...
6,8774,The Wok Shop,True,False,4.1,270,Koramangala 1st Block,Quick Bites,"Momos, Rice Noodles, Kung Pao Chicken, Noodle,...",Korean,500.0,tried asian bao burger penut souce really real...
7,28965,Kuttanadu,True,False,2.8,261,HSR,Casual Dining,"Pothichoru, Kerala Parotta, Beef Fry, Vegetabl...",Chinese,700.0,kuttanadu probably best kerala restaurant part...
8,21198,Rajvardhan Foods,True,False,3.9,315,Jayanagar,Quick Bites,"Vada Pav, Misal Pav, Sabudana Vada, Dabeli, Pa...",Maharashtrian,300.0,poha amazing minimal oil wide range local chut...
9,2550,Energy Addaa,True,False,4.2,64,Basavanagudi,Quick Bites,"Pizza, Nachos, Salads, Peri Peri Fries, Browni...",Healthy Food,500.0,place two foodie friendsnnthis situated opposi...


#### MODEL ON TRAINING DATASET

In [175]:
train_df.set_index('name', inplace=True)

In [176]:
indices = pd.Series(train_df.index)

In [177]:
train_df[train_df['location'] == 'City Market']['cuisines']#.unique()

name
B Bhagat Tarachand    Chinese
Pai Vihar             Chinese
Name: cuisines, dtype: object

In [ ]:
##### CHOOSING THE RESTAURANT

In [191]:
def user_recommendation():
    # User input
    unique_loc = train_df['location'].unique()
    print("Choose Location:")
    for i, loc in enumerate(unique_loc, start=1):
        print(f"{i}. {loc}")
        
    location = input("Where are you located? ")
    
    unique_cuisines_df = train_df[train_df['location'] == location]['cuisines']
    unique_cuisines = unique_cuisines_df.unique()
    if unique_cuisines_df.empty:
        print("Choose location from the list")
        return
    
    print("Choose Cuisine:")
    for i, loc in enumerate(unique_cuisines, start=1):
        print(f"{i}. {loc}")
        
    cuisine = input("What are you craving for? ")
    
    Rest_Name_df = train_df[(train_df['location'] == location) & (train_df['cuisines'] == cuisine)]
    if Rest_Name_df.empty:
        print("Choose Cuisine from the list")
        return
    Rest_Name = Rest_Name_df['rest_type'].unique()
    print("Choose Restaurant:")
    for i, loc in enumerate(Rest_Name, start=1):
        print(f"{i}. {loc}")
    
    rest_type = input("Preferred restaurant type: ")
    
    rest_type = rest_type.lstrip()
    rest_type = rest_type.rstrip()
    location= location.lstrip()
    location= location.rstrip()
    cuisine=cuisine.lstrip()
    cuisine=cuisine.rstrip()
    

    # Filter the dataframe based on user preferences
    filtered_df = train_df[(train_df['location'] == location) & (train_df['cuisines'] == cuisine) & (train_df['rest_type'] == rest_type)]
    #filtered_df = Rest_Name_df[Rest_Name_df['rest_type'] == rest_type]
    
     
    if filtered_df.empty:
                    print("4")
                    print("No Restaurants Found")
                    return
    num_rows = filtered_df.shape[0]        
    if num_rows == 1:
        
        name =filtered_df.index.tolist()
        print(name)
        recommendations_df = pd.DataFrame({'Restaurant': name, 'Rating': filtered_df['rate']})
        recommendations_df = recommendations_df.reset_index(drop=True)
    
    else:        
    # Creating tf-idf matrix for reviews
        tfidf_reviews = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
        tfidf_reviews_matrix = tfidf_reviews.fit_transform(filtered_df['reviews_list'])
    
    # Compute cosine similarity for reviews
        cosine_reviews = cosine_similarity(tfidf_reviews_matrix, tfidf_reviews_matrix)
    #print("cosine_reviews : \n" , cosine_reviews)
    # Find the index of the restaurant entered
        idx = filtered_df.index[0]  # Use the first index from the filtered dataframe
    #print("idx : " , idx)
        unique_indexes = filtered_df.index.unique()
    #print(unique_indexes)
    # Find the restaurants with a similar cosine-sim value for reviews and order them from biggest number
        if idx in unique_indexes : #filtered_df.index.unique():
            idx = unique_indexes.get_loc(idx) #filtered_df.index.get_loc(idx)
            sim_scores_reviews = list(enumerate(cosine_reviews[idx]))
            sim_scores_reviews = sorted(list(sim_scores_reviews), key=lambda x: x[1], reverse=True)
        # Extract top 5 restaurant indexes with a similar cosine-sim value (excluding the selected restaurant itself)
            top5_indexes_reviews = [i[0] for i in sim_scores_reviews[1:6]]
        
        # Names of the top 5 restaurants based on reviews
            recommend_restaurants_reviews = list(filtered_df.index[top5_indexes_reviews].unique())
        #print('Rest : ',recommend_restaurants_reviews)
        # Ratings of the top 5 restaurants based on reviews
            recommend_ratings_reviews = [np.unique(filtered_df.loc[restaurant]['rate']) for restaurant in recommend_restaurants_reviews]

        #recommend_ratings_reviews = [filtered_df.loc[restaurant]['rate'].unique() for restaurant in recommend_restaurants_reviews]
        #print("Rate : ",recommend_ratings_reviews)
        if not recommend_restaurants_reviews:
            print("5")
            print("No Restaurants Found")
            return
    
    # Create a DataFrame to store the recommendations and ratings
        recommendations_df = pd.DataFrame({'Restaurant': recommend_restaurants_reviews, 'Rating': recommend_ratings_reviews})
    
    print("Restaurants you may also like: ")
    print(recommendations_df)
    
    # User selects a restaurant
    selected_restaurant = input("Select a restaurant from the list above: ")
    
    if selected_restaurant not in recommendations_df['Restaurant'].values:
                print("Invalid restaurant selection")
                return
    
    # Get the index of the selected restaurant
    selected_index = recommendations_df[recommendations_df['Restaurant'] == selected_restaurant].index[0]
    # Get the recommended dishes for the selected restaurant
    selected_recommend_dishes = filtered_df.loc[recommendations_df.loc[selected_index]['Restaurant']]['dish_liked']
    selected_recommend_dishes_series = pd.Series(selected_recommend_dishes)
    unique_dishes = selected_recommend_dishes_series.unique()

    #selected_recommend_dishes_list = list(selected_recommend_dishes)
    #unique_dishes = list(set(selected_recommend_dishes_list))
    print("Dishes you will like in", selected_restaurant, ":")
    print(unique_dishes)
    
user_recommendation()

Choose Location:
1. Bannerghatta Road
2. Kalyan Nagar
3. Ulsoor
4. Sarjapur Road
5. Jayanagar
6. Koramangala 1st Block
7. HSR
8. Basavanagudi
9. Koramangala 4th Block
10. Basaveshwara Nagar
11. Brookefield
12. JP Nagar
13. BTM
14. Residency Road
15. Bellandur
16. Lavelle Road
17. Marathahalli
18. Koramangala 6th Block
19. Koramangala 7th Block
20. Banaswadi
21. Church Street
22. Electronic City
23. Brigade Road
24. Domlur
25. MG Road
26. Koramangala 5th Block
27. East Bangalore
28. Frazer Town
29. Malleshwaram
30. Indiranagar
31. Banashankari
32. Richmond Road
33. Infantry Road
34. New BEL Road
35. Kammanahalli
36. Rajajinagar
37. Race Course Road
38. Koramangala 8th Block
39. Ejipura
40. Wilson Garden
41. Kumaraswamy Layout
42. ITPL Main Road, Whitefield
43. St. Marks Road
44. Old Airport Road
45. Cunningham Road
46. Whitefield
47. Shivajinagar
48. Jeevan Bhima Nagar
49. Koramangala 2nd Block
50. Sanjay Nagar
51. Thippasandra
52. Koramangala 3rd Block
53. Varthur Main Road, Whitefield

In [179]:
unique_cuisines = train_df[train_df['location'] == 'Sankey Road']
Rest_Name = train_df[(train_df['location'] == 'Sankey Road') & (train_df['cuisines'] == 'Continental')]['rest_type'].unique()
filtered_df = train_df[(train_df['location'] == 'Ulsoor') & (train_df['cuisines'] == 'Chinese') & 
                           (train_df['rest_type'] == 'Fine Dining')]
filtered_df

,rest_id,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list
name,,,,,,,,,,,
Mikusu Conrad Bengaluru,19137,False,True,4.1,80,Ulsoor,Fine Dining,"Sushi, Noodles",Chinese,3.0,place known sushi thai food really enjoyed tha...


In [169]:
print(train_df.columns)

Index(['rest_id', 'online_order', 'book_table', 'rate', 'votes', 'location',
       'rest_type', 'dish_liked', 'cuisines', 'cost', 'reviews_list'],
      dtype='object')


In [65]:
train_df['location'].unique()

array(['Bannerghatta Road', 'Kalyan Nagar', 'Ulsoor', 'Sarjapur Road',
       'Jayanagar', 'Koramangala 1st Block', 'HSR', 'Basavanagudi',
       'Koramangala 4th Block', 'Basaveshwara Nagar', 'Brookefield',
       'JP Nagar', 'BTM', 'Residency Road', 'Bellandur', 'Lavelle Road',
       'Marathahalli', 'Koramangala 6th Block', 'Koramangala 7th Block',
       'Banaswadi', 'Church Street', 'Electronic City', 'Brigade Road',
       'Domlur', 'MG Road', 'Koramangala 5th Block', 'East Bangalore',
       'Frazer Town', 'Malleshwaram', 'Indiranagar', 'Banashankari',
       'Richmond Road', 'Infantry Road', 'New BEL Road', 'Kammanahalli',
       'Rajajinagar', 'Race Course Road', 'Koramangala 8th Block',
       'Ejipura', 'Wilson Garden', 'Kumaraswamy Layout',
       'ITPL Main Road, Whitefield', 'St. Marks Road', 'Old Airport Road',
       'Cunningham Road', 'Whitefield', 'Shivajinagar',
       'Jeevan Bhima Nagar', 'Koramangala 2nd Block', 'Sanjay Nagar',
       'Thippasandra', 'Koramangala 